# Set up

In [29]:
import tiktoken
import pandas as pd
import numpy as np
from sklearn.metrics import multilabel_confusion_matrix, label_ranking_average_precision_score, label_ranking_loss, precision_score, recall_score, f1_score, accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
import seaborn as sns

# Import Azure OpenAI
import os
from langchain_openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI
import json

In [30]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import warnings
warnings.filterwarnings('ignore')

In [31]:
import getpass
import os

if "MISTRAL_API_KEY" not in os.environ:
    os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")

In [ ]:
MISTRAL_API_KEY = ""

In [33]:
# Ensure your mistral credentials are configured
import getpass
import os
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(
    model="ministral-8b-latest",
    temperature=0,
    max_retries=2,
    # other params...
)

## Préparation des fonctions utiles

In [34]:
categories = [
    "Gaz à effet de serre",
    "Elevage et utilisation des terres",
    "Pêche et chasse intensives",
    "Pollution plastique",
    "Déforestation",
    "Surconsommation",
    "Catastrophes naturelles",
    "Réchauffement climatique/canicules",
    "Sécheresse",
    "Couche d'ozone",
    "Feu de forêt",
    "Tensions alimentaires/famines",
    "Perte eau douce",
    "Hausse des océans et fonte des glaces",
    "Conséquence sociale",
    "Acidification des océans",
    "Biodiversité",
    "Pollution",
    "Energie renouvelable et nucléaire",
    "Transports décarbonés",
    "Engagements politiques et entreprises",
    "Activisme écologique",
    "Solutions innovantes"
    "Comportement de consommation",
    "Reforestation",

]

In [35]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

def classification(text, llm, categories):
    """
    Function that classifies a text into a given list of categories

    Arguments:
    - text: str, the text to classify
    - llm: AzureChatOpenAI, the language model to use
    - categories: list, the list of categories to classify the text into

    Returns:
    - str, the category that best fits the text
    """
    try: 
        prompt_message = f"""  
            Donne uniquement la classe, parmi {categories}, qui correspond le mieux au texte donné
            ## Règles :
            Ne répondre qu'avec un seul terme dans {categories} .
            ## Exemple :
            Texte : "Les océans absorbent trop de CO₂, ce qui perturbe la vie marine."
            Réponse : {{"category": "Acidification des océans"}}

            """
        user_message = f"""
            Voici le texte à classifier:
            {text}
            """
        messages = [
        SystemMessage(content=prompt_message)
        ]
        messages.append(
            HumanMessage(content=user_message)
        )
        response = llm(messages)

        return response.content
    except Exception as e:
        print(e)
        return None

text2 = """
La France offre l’un des plus vastes domaines skiables au monde avec 350 stations de sport d’hiver. Mais combien en restera-t-il dans 20 ans ? Avec le réchauffement climatique, la neige est plus rare en moyenne altitude, les saisons sont de plus en plus courtes et certains sites ne sont plus rentables.Il faut grimper à 1 200 mètres d’altitude, à travers les sapins du Massif central, pour le trouver. À quelques encablures d’une petite station de ski, à Chalmazel (Loire), se trouve un village vacances fantôme. Cela fait 20 ans que la résidence a fermé, et que le bâtiment rempli d’amiante est abandonné. \Il y a eu énormément de souvenirs et d’ambiance\", se souvient le maire Valéry Gouttefarde, entre dépit et nostalgie. Dans les années 70, chaque bout de montagne devenait un eldorado. Des stations de ski trop petites ou trop peu enneigées Face à la concurrence des grosses stations des Alpes, la résidence difficile à chauffer est devenue passée de mode, impossible à rentabiliser. La seule solution aujourd’hui, c’est une coûteuse démolition, estimée à plus d’un million d’euros. \"On construisait sans se soucier du recyclage et du devenir de ces bâtiments\", concède le maire. Et c’est loin d’être le seul site en France à démanteler. Villages vacances vides, remontées mécaniques désaffectées... Il y aurait en tout plus de 150 stations de ski fantômes, trop petites pour survivre ou trop peu enneigées. Le site de Moutainwilderness Le site d'Installations obsolètes"
"""

classification(text2, llm, categories)

'Réchauffement climatique/canicules'

In [36]:
Exemple = """
            Texte : "Les émissions de CO₂ ont atteint un niveau record cette année."
            Réponse JSON:
            {{
                "Classe_1": "Gaz à effet de serre",
                "Classe_2": "Elevage et utilisation des terres",
                "Classe_3": "Surconsommation"
            }}
            """

In [37]:
#solu chatgpt

import time
import requests
import json 
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

def classification_up_to_k(text, llm, categories, k=3, retry_attempts=5, base_delay=10):
    """
    Function that classifies a text into a given list of k categories
    """
    try:
        output_format = {f"Classe_{i}": "categorie_{i}" for i in range(1, k + 1)}
        prompt_message = f"""
            Donne les {k} classes, parmi {categories}, qui correspondent le mieux au texte donné, de la plus probable à la moins probable.
            ## Règles:
            Pour chacune des {k} prédictions, ne répondre qu'avec un seul terme dans {categories}.
            Réponds strictement avec un dictionnaire du format JSON suivant: {output_format} où les valeurs sont des listes de catégories.
            Les classes sont ordonnées de la plus probable à la moins probable.
            
        """
        user_message = f"Voici le texte à classifier:\n{text}"
        
        messages = [SystemMessage(content=prompt_message), HumanMessage(content=user_message)]

        for attempt in range(retry_attempts):
            try:
                response = llm(messages).content
                response = "{" + response.partition("{")[2].partition("}")[0] +"}"
                try:
                    response_json = json.loads(response)  # Convertir la réponse en dictionnaire JSON
                    return response_json  # Retourner directement le JSON valide
                except json.JSONDecodeError:
                    print("Erreur : la réponse du modèle n'est pas un JSON valide.")
                    return None
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429:
                    retry_after = int(e.response.headers.get('Retry-After', base_delay)) if 'Retry-After' in e.response.headers else base_delay
                    print(f"Rate limit exceeded. Retrying in {retry_after} seconds...")
                    time.sleep(retry_after)
                    base_delay *= 2  # Augmenter le délai exponentiellement
                else:
                    print(f"Unexpected HTTP error: {e.response.status_code}, {e.response.text}")
                    return None
            except Exception as e:
                print(f"Unexpected error: {e}")
                return None
        print("Max retry attempts reached. Returning None.")
        return None
    except Exception as e:
        print(f"Outer exception: {e}")
        return None


In [38]:
def classification_up_to_k_COT(text, llm, categories, k=3, retry_attempts=5, base_delay=10):
    """
    Function that classifies a text into a given list of k categories with a reasoning
    """
    try:
        output_format = {f"Classe_{i}": "categorie_{i}" for i in range(1, k + 1)}
        prompt_message = f"""
            Donne les {k} classes, parmi {categories}, qui correspondent le mieux au texte donné, de la plus probable à la moins probable. Avant cela explique ton raisonnement.
            ## Règles:
            Commence par un "Raisonnement" : ou tu expliques pourquoi tu as choisi ces classes.
            Pour chacune des {k} prédictions, ne répondre qu'avec un seul terme dans {categories}.
            Réponds pour les classes strictement avec un dictionnaire du format JSON suivant: {output_format} où les valeurs sont des listes de catégories
            Les classes sont ordonnées de la plus probable à la moins probable.
        """
        user_message = f"Voici le texte à classifier:\n{text}"
        
        messages = [SystemMessage(content=prompt_message), HumanMessage(content=user_message)]

        for attempt in range(retry_attempts):
            try:
                response = llm(messages).content
                response = "{" + response.partition("{")[2].partition("}")[0] +"}"
                try:
                    response_json = json.loads(response)  # Convertir la réponse en dictionnaire JSON
                    return response_json  # Retourner directement le JSON valide
                except json.JSONDecodeError:
                    print(response)
                    print("Erreur : la réponse du modèle n'est pas un JSON valide.")
                    return None
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429:
                    retry_after = int(e.response.headers.get('Retry-After', base_delay)) if 'Retry-After' in e.response.headers else base_delay
                    print(f"Rate limit exceeded. Retrying in {retry_after} seconds...")
                    time.sleep(retry_after)
                    base_delay *= 2  # Augmenter le délai exponentiellement
                else:
                    print(f"Unexpected HTTP error: {e.response.status_code}, {e.response.text}")
                    return None
            except Exception as e:
                print(f"Unexpected error: {e}")
                return None
        print("Max retry attempts reached. Returning None.")
        return None
    except Exception as e:
        print(f"Outer exception: {e}")
        return None


In [39]:
def classification_up_to_k_CARP(text, llm, categories, k=3, retry_attempts=5, base_delay=10):
    """
    Function that classifies a text into a given list of k categories with a reasoning
    """
    try:
        output_format = {f"Classe_{i}": "categorie_{i}" for i in range(1, k + 1)}
        prompt_message = f"""
            Donne les {k} classes, parmi {categories}, qui correspondent le mieux au texte donné, de la plus probable à la moins probable. Avant cela explique ton raisonnement.
            ## Règles:
            Commence par Indices une liste d'INDICES pris dans le texte fourni (c'est-à-dire de mots-clés, de phrases, d'informations contextuelles, de relations sémantiques, de sens sémantique, de tonalités, de références) qui soutiennent la détermination des classes.
            Puis fais un Raisonnement déduire le processus de RAISONNEMENT diagnostique à partir des prémisses (c'est-à-dire les indices, l'entrée) qui soutiennent la détermination des classes du texte (limiter le nombre de mots à 130).
            Pour chacune des {k} prédictions, ne répondre qu'avec un seul terme dans {categories}.
            Réponds pour les classes strictement avec un dictionnaire du format JSON suivant: {output_format} où les valeurs sont des listes de catégories.
            Les classes sont ordonnées de la plus probable à la moins probable.
        """
        user_message = f"Voici le texte à classifier:\n{text}"
        
        messages = [SystemMessage(content=prompt_message), HumanMessage(content=user_message)]

        for attempt in range(retry_attempts):
            try:
                response = llm(messages).content
                response = "{" + response.partition("{")[2].partition("}")[0] +"}"
                try:
                    response_json = json.loads(response)  # Convertir la réponse en dictionnaire JSON
                    return response_json  # Retourner directement le JSON valide
                except json.JSONDecodeError:
                    print(response)
                    print("Erreur : la réponse du modèle n'est pas un JSON valide.")
                    return None
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429:
                    retry_after = int(e.response.headers.get('Retry-After', base_delay)) if 'Retry-After' in e.response.headers else base_delay
                    print(f"Rate limit exceeded. Retrying in {retry_after} seconds...")
                    time.sleep(retry_after)
                    base_delay *= 2  # Augmenter le délai exponentiellement
                else:
                    print(f"Unexpected HTTP error: {e.response.status_code}, {e.response.text}")
                    return None
            except Exception as e:
                print(f"Unexpected error: {e}")
                return None
        print("Max retry attempts reached. Returning None.")
        return None
    except Exception as e:
        print(f"Outer exception: {e}")
        return None


# Tests

In [40]:
text3 = """
"en france, les transports sont responsables de 32% des émissions de gaz à effet de serre" : françois gemenne veut décarboner aussi nos routes cette semaine on s’intéresse à l’alliance pour la décarbonation de la route, une initiative lancée, le 12 décembre 2023, par françois gemenne avec plusieurs collègues et universitaires. c’est un regroupement de chercheurs, d’industriels et de collectivités, qui veulent travailler à décarboner la route, et à mettre en œuvre des solutions concrètes pour réduire les émissions de gaz à effet de serre provoquées par le trafic routier. franceinfo : pourquoi la route ? pourquoi c’est important ? parce que c’est la principale source d’émissions de gaz à effet de serre en france. en france, selon le haut conseil pour le climat, les émissions du secteur des transports sont responsables de 32% des émissions. et l’immense majorité de ces émissions, elles sont provoquées par le transport routier : 94% des émissions de transports en france, ce sont les voitures, les camions et les véhicules utilitaires. et surtout c’est un secteur dont les émissions ne baissent guère, contrairement à d’autres secteurs comme l’industrie et le bâtiment. selon l’agence européenne de l’environnement, les émissions du secteur des transports seront en 2030 supérieures de 10% à leur niveau de 1990. il y a la voiture électrique, pourtant… bien sûr. et c’est important, mais ça ne suffit pas. parce qu’il y a aussi la question des infrastructures, de la route elle-même. si on n’équipe pas les infrastructures, si on ne permet pas davantage de report modal, par exemple, si on n’investit pas davantage dans le transport public, on n’y arrivera pas. "si on ne s’attaque pas à la route, on ne parviendra jamais à atteindre notre objectif de réduction des émissions de 55% d’ici 2030, au niveau national." et donc, comment peut-on faire ? le problème, c’est que dès qu’on s’attache à la voiture, les français ont l’impression qu’on s’attaque à leur liberté individuelle, et on voit que le soutien des français aux mesures qui touchent à la voiture est de plus en plus faible. ce que nous disons, c’est qu’il faut décarboner la route elle-même, et pas seulement les véhicules qui roulent dessus. mais est-ce que la solution ce n’est pas le train, tout simplement ? le report modal, vers le train notamment, ou vers le vélo pour les courts trajets, fait évidemment partie de la solution. mais aujourd’hui, l’écrasante majorité des déplacements du quotidien continuent à se faire en voiture, et même dans des voitures dans lesquelles on voyage seul. donc il faut absolument s’attaquer à ces émissions-là. comment peut-on faire ? quelles sont les solutions ? la bonne nouvelle, c’est qu’il y a énormément de solutions, et que ces solutions ne demandent qu’à être mises en œuvre. il y a la transformation de nos routes et de nos autoroutes, mais aussi des solutions de transport collectif : par exemple des cars express dans des régions rurales ou semi-rurales. ou des lignes de co-voiturage, pour permettre aux gens de partager leur voiture. il y a plein de choses qu’on peut faire. des solutions techniques ou technologiques, mais aussi des solutions d’usages, de pratiques, d’aménagements urbains ou d’aménagements routiers… pourquoi est-ce qu’on ne le fait pas, alors ? "la route a souvent été le parent pauvre de nos politiques de décarbonation." c’est pour ça que nous lançons cette alliance : pour faire travailler ensemble tous les gens qui ont des solutions et qui veulent les mettre en œuvre. des constructeurs automobiles, des concessionnaires d’autoroutes, des professionnels du co-voiturage, des exploitants de transport collectif, des entreprises du btp, des assurances, en lien avec les collectivités, bien entendu. l’idée, c’est de proposer des solutions concrètes, et de voir ensuite avec les pouvoirs publics comment il est possible de les mettre en œuvre. et on invite tous ceux qui sont intéressés, qui ont des solutions à proposer, à nous rejoindre.
"""
classification_up_to_k(text3, llm, categories)

{'Classe_1': 'Gaz à effet de serre',
 'Classe_2': 'Transports décarbonés',
 'Classe_3': 'Engagements politiques et entreprises'}

In [41]:
text4 = """
agriculture : changer les habitudes face au réchauffement climatique l’agriculture est responsable d’un quart des émissions de gaz à effet de serre, constate le giec (groupement d’experts intergouvernemental sur l’évolution du climat). les experts préconisent une agriculture raisonnée et de modifier notre alimentation. faut-il manger moins de viande ? dans son rapport de 1 200 pages, le giec (groupement d’experts intergouvernemental sur l’évolution du climat) préconise de revoir les pratiques agricoles car l’agriculture émet 24 % des gaz à effet de serre. "on constate que les activités humaines se sont déployées sur les trois quarts des sols de notre planète. au cours des dernières décennies, l'humanité a intensifié l’utilisation des terres et l’exploitation de l’eau potable", constate marc chardonnens, directeur de l’office fédéral de l’environnement (ofev) réduire la viande la demande va augmenter avec la population. la planète va passer de 7 milliards de personnes en 2019 à 9,8 milliards en 2050. de plus en plus de bouches à nourrir donc, alors que la terre est à bout de souffle. des pistes sont étudiées : rendre à la forêt des zones d'élevage de ruminants ou changer de régime alimentaire comme réduire la viande et les produits laitiers. il ne faudrait pas dépasser 33 kg par an. or, en france, nous en consommons 260 kg par an.
"""

In [42]:
time.sleep(4)
classification(text4, llm, categories)

'"Elevage et utilisation des terres"'

In [43]:
time.sleep(4)
classification_up_to_k(text4, llm, categories)

{'Classe_1': 'Elevage et utilisation des terres',
 'Classe_2': 'Gaz à effet de serre',
 'Classe_3': 'Surconsommation'}

In [44]:
time.sleep(4)
classification_up_to_k_COT(text4, llm, categories)

{'Classe_1': 'Elevage et utilisation des terres',
 'Classe_2': 'Gaz à effet de serre',
 'Classe_3': 'Conséquence sociale'}

In [45]:
time.sleep(4)
classification_up_to_k_CARP(text4, llm, categories)

{'Classe_1': 'Gaz à effet de serre',
 'Classe_2': 'Elevage et utilisation des terres',
 'Classe_3': 'Surconsommation'}

## Inférence Mistral

### Classification multi-label avec score de confiance entre 0 et 1

On demande ici à Mistral de prédire 3 classes avec un degré de confiance pour chacune. Ce degré de confiance est un nombre entre 0 et 1 et peut être vu comme une probabilité d'appartenance à chaque classe.

In [46]:
path_df = "./annotation_sous_thematiques.csv"

In [47]:
examples_raw = [""" Texte:
                agriculture : changer les habitudes face au réchauffement climatique l’agriculture est responsable d’un quart des émissions de gaz à effet de serre, constate le giec (groupement d’experts intergouvernemental sur l’évolution du climat). les experts préconisent une agriculture raisonnée et de modifier notre alimentation. faut-il manger moins de viande ? dans son rapport de 1 200 pages, le giec (groupement d’experts intergouvernemental sur l’évolution du climat) préconise de revoir les pratiques agricoles car l’agriculture émet 24 % des gaz à effet de serre. "on constate que les activités humaines se sont déployées sur les trois quarts des sols de notre planète. au cours des dernières décennies, l'humanité a intensifié l’utilisation des terres et l’exploitation de l’eau potable", constate marc chardonnens, directeur de l’office fédéral de l’environnement (ofev) réduire la viande la demande va augmenter avec la population. la planète va passer de 7 milliards de personnes en 2019 à 9,8 milliards en 2050. de plus en plus de bouches à nourrir donc, alors que la terre est à bout de souffle. des pistes sont étudiées : rendre à la forêt des zones d'élevage de ruminants ou changer de régime alimentaire comme réduire la viande et les produits laitiers. il ne faudrait pas dépasser 33 kg par an. or, en france, nous en consommons 260 kg par an.
Réponse :
{
  "Classe_1": "Gaz à effet de serre",
  "Classe_2": "Elevage et utilisation des terres",
  "Classe_3": "Comportement de consommation"
}
""","""2""","""3""",""""""]
examples_COT = [""" Texte:
                agriculture : changer les habitudes face au réchauffement climatique l’agriculture est responsable d’un quart des émissions de gaz à effet de serre, constate le giec (groupement d’experts intergouvernemental sur l’évolution du climat). les experts préconisent une agriculture raisonnée et de modifier notre alimentation. faut-il manger moins de viande ? dans son rapport de 1 200 pages, le giec (groupement d’experts intergouvernemental sur l’évolution du climat) préconise de revoir les pratiques agricoles car l’agriculture émet 24 % des gaz à effet de serre. "on constate que les activités humaines se sont déployées sur les trois quarts des sols de notre planète. au cours des dernières décennies, l'humanité a intensifié l’utilisation des terres et l’exploitation de l’eau potable", constate marc chardonnens, directeur de l’office fédéral de l’environnement (ofev) réduire la viande la demande va augmenter avec la population. la planète va passer de 7 milliards de personnes en 2019 à 9,8 milliards en 2050. de plus en plus de bouches à nourrir donc, alors que la terre est à bout de souffle. des pistes sont étudiées : rendre à la forêt des zones d'élevage de ruminants ou changer de régime alimentaire comme réduire la viande et les produits laitiers. il ne faudrait pas dépasser 33 kg par an. or, en france, nous en consommons 260 kg par an.
Réponse :
    Gaz à effet de serre : Le texte souligne que l'agriculture est responsable de 24 % des émissions de gaz à effet de serre, ce qui est un indicateur clair de cette catégorie.
    Elevage et utilisation des terres : Le texte discute de l'utilisation intensive des terres pour l'élevage et l'agriculture, ce qui correspond directement à cette catégorie.
    Comportement de consommation : Le texte propose de modifier les habitudes alimentaires, notamment en réduisant la consommation de viande, ce qui relève du comportement de consommation.
{
  "Classe_1": "Gaz à effet de serre",
  "Classe_2": "Elevage et utilisation des terres",
  "Classe_3": "Comportement de consommation"
}
""","""2""","""3""",""""""]
examples_CARP = ["""
Texte:
    agriculture : changer les habitudes face au réchauffement climatique l’agriculture est responsable d’un quart des émissions de gaz à effet de serre, constate le giec (groupement d’experts intergouvernemental sur l’évolution du climat). les experts préconisent une agriculture raisonnée et de modifier notre alimentation. faut-il manger moins de viande ? dans son rapport de 1 200 pages, le giec (groupement d’experts intergouvernemental sur l’évolution du climat) préconise de revoir les pratiques agricoles car l’agriculture émet 24 % des gaz à effet de serre. "on constate que les activités humaines se sont déployées sur les trois quarts des sols de notre planète. au cours des dernières décennies, l'humanité a intensifié l’utilisation des terres et l’exploitation de l’eau potable", constate marc chardonnens, directeur de l’office fédéral de l’environnement (ofev) réduire la viande la demande va augmenter avec la population. la planète va passer de 7 milliards de personnes en 2019 à 9,8 milliards en 2050. de plus en plus de bouches à nourrir donc, alors que la terre est à bout de souffle. des pistes sont étudiées : rendre à la forêt des zones d'élevage de ruminants ou changer de régime alimentaire comme réduire la viande et les produits laitiers. il ne faudrait pas dépasser 33 kg par an. or, en france, nous en consommons 260 kg par an.
Réponse :

Indices:
"l'agriculture est responsable d’un quart des émissions de gaz à effet de serre","les activités humaines se sont déployées sur les trois quarts des sols de notre planète","réduire la viande et les produits laitiers","rendre à la forêt des zones d'élevage de ruminants","l'humanité a intensifié l’utilisation des terres et l’exploitation de l’eau potable"
Raisonnement:
Gaz à effet de serre : Le texte mentionne explicitement que l'agriculture est responsable de 24 % des émissions de gaz à effet de serre, ce qui justifie cette catégorie.
Elevage et utilisation des terres : Le texte parle de l'utilisation intensive des terres pour l'élevage et l'agriculture, soulignant l'impact sur les sols et l'exploitation des ressources
Comportement de consommation : Le texte propose de modifier les habitudes alimentaires, notamment en réduisant la consommation de viande et de produits laitiers, ce qui relève directement du comportement de consommation.

Voici les classes correspondantes :

{
  "Classe_1": "Gaz à effet de serre",
  "Classe_2": "Elevage et utilisation des terres",
  "Classe_3": "Comportement de consommation"
}
""","""2""","""3""",""""""]

In [ ]:
def get_prompt_message(output_format,k, categories ,prompt_method,n_shots,recursive=False,previous_answer = ""):
    if prompt_method == "raw":
        prompt_message = f"""
            Donne les classes, parmi {categories}, qui correspondent le mieux au texte donné.
            ## Règles:
            Pour chacune des {k} prédictions, ne répondre qu'avec un seul terme dans {categories}
            Réponds pour les classes strictement avec un dictionnaire du format JSON suivant: {output_format} où les valeurs sont des listes de catégories.
            Les classes sont ordonnées de la plus probable à la moins probable.
        """ 
        prompt_message +=  '\n'.join(examples_raw[:n_shots])
    elif prompt_method == "COT":
        prompt_message = f"""
            Donne les {k} classes, parmi {categories}, qui correspondent le mieux au texte donné, de la plus probable à la moins probable. Avant cela explique ton raisonnement.
            ## Règles:
            Fais un Raisonnement déduire le processus de RAISONNEMENT diagnostique à partir des prémisses (c'est-à-dire les indices, l'entrée) qui soutiennent la détermination des classes du texte (limiter le nombre de mots à 130).
            Pour chacune des {k} prédictions, ne répondre qu'avec un seul terme dans {categories}.
            Réponds pour les classes strictement avec un dictionnaire du format JSON suivant: {output_format} où les valeurs sont des listes de catégories.
            Les classes sont ordonnées de la plus probable à la moins probable.
        """
        prompt_message +=  '\n'.join(examples_COT[:n_shots])
    elif prompt_method == "CARP":
        prompt_message = f"""
            Donne les {k} classes, parmi {categories}, qui correspondent le mieux au texte donné, de la plus probable à la moins probable. Avant cela explique ton raisonnement.
            ## Règles:
            Commence par Indices une liste d'INDICES pris dans le texte fourni (c'est-à-dire de mots-clés, de phrases, d'informations contextuelles, de relations sémantiques, de sens sémantique, de tonalités, de références) qui soutiennent la détermination des classes.
            Puis fais un Raisonnement déduire le processus de RAISONNEMENT diagnostique à partir des prémisses (c'est-à-dire les indices, l'entrée) qui soutiennent la détermination des classes du texte (limiter le nombre de mots à 130).
            Pour chacune des {k} prédictions, ne répondre qu'avec un seul terme dans {categories}.
            Réponds pour les classes strictement avec un dictionnaire du format JSON suivant: {output_format} où les valeurs sont des listes de catégories.
            Les classes sont ordonnées de la plus probable à la moins probable.
        """
        prompt_message +=  '\n'.join(examples_CARP[:n_shots])
    
    if recursive :
        prompt_message += '\n' + f"{previous_answer}"
    return prompt_message

In [49]:
def classification_up_to_k_general(text, llm,categories,prompt_method = "raw",n_shots=0,k = 3,retry_attempts=5, base_delay=10):
    """
    Function that classifies a text into a given list of k categories with a reasoning
    """
    try:
        output_format = {f"Classe_{i}": "categorie_{i}" for i in range(1, k + 1)}
        prompt_message = get_prompt_message(output_format,k, categories ,prompt_method,n_shots)
        user_message = f"Voici le texte à classifier:\n{text}"
        messages = [SystemMessage(content=prompt_message), HumanMessage(content=user_message)]

        for attempt in range(retry_attempts):
            try:
                response = llm(messages).content
                response = "{" + response.partition("{")[2].partition("}")[0] +"}"
                try:
                    response_json = json.loads(response)  # Convertir la réponse en dictionnaire JSON
                    return response_json  # Retourner directement le JSON valide
                except json.JSONDecodeError:
                    print(response)
                    print("Erreur : la réponse du modèle n'est pas un JSON valide.")
                    return None
            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429:
                    retry_after = int(e.response.headers.get('Retry-After', base_delay)) if 'Retry-After' in e.response.headers else base_delay
                    print(f"Rate limit exceeded. Retrying in {retry_after} seconds...")
                    time.sleep(retry_after)
                    base_delay *= 2  # Augmenter le délai exponentiellement
                else:
                    print(f"Unexpected HTTP error: {e.response.status_code}, {e.response.text}")
                    return None
            except Exception as e:
                print(f"Unexpected error: {e}")
                return None
        print("Max retry attempts reached. Returning None.")
        return None
    except Exception as e:
        print(f"Outer exception: {e}")
        return None

In [50]:
def classification_up_to_k_recursive(text, llm,categories,prompt_method = "raw",n_shots=0,k = 3,retry_attempts=5, base_delay=10 , n_iter = 1):
    prompt = classification_up_to_k_general(text, llm,categories,prompt_method,n_shots,k,retry_attempts, base_delay)
    for i in range(n_iter): 
        time.sleep(4) 
        output_format = {f"Classe_{i}": "categorie_{i}" for i in range(1, k + 1)}
        prompt_message = get_prompt_message(output_format,k, categories ,prompt_method,n_shots,recursive=True,previous_answer = prompt)
        user_message = f"Voici le texte à classifier:\n{text}"
        messages = [SystemMessage(content=prompt_message), HumanMessage(content=user_message)]

        for attempt in range(retry_attempts):
            try:
                response = llm(messages).content
                response = "{" + response.partition("{")[2].partition("}")[0] +"}"
            except:
                return None
        
        prompt_message += "\n" + response
    try:
        response_json = json.loads(response)  # Convertir la réponse en dictionnaire JSON
        return response_json  # Retourner directement le JSON valide
    except json.JSONDecodeError:
        print(response)
        print("Erreur : la réponse du modèle n'est pas un JSON valide.")
        return None

In [51]:
classification_up_to_k_recursive(text4, llm, categories, prompt_method = "COT", n_shots=0, k = 3, retry_attempts=5, base_delay=10 , n_iter = 3)

{'Classe_1': 'Gaz à effet de serre',
 'Classe_2': 'Elevage et utilisation des terres',
 'Classe_3': 'Surconsommation'}

In [52]:
def get_df_pred(path_df = "./annotation_sous_thematiques.csv" ,classification = classification_up_to_k_general,recursive = False,prompt_method = "raw",model="ministral8b",n_shots = 0):
    df = pd.read_csv(path_df)
    df = df.head(400)
    df = df[df['label']==1]
    df.shape
    df["prediction_label"] = None
    events_count = df.shape[0]
    k = 0
    for idx, row in df.iterrows():
        df.at[idx, "prediction_label"] = classification(row["description"], llm, categories,prompt_method,n_shots)
        time.sleep(4)  # Attendre 4 secondes entre chaque requête   
        
        if k % 28 == 0 and k > 0:
            finished = 100*(k/events_count)
            print('Finished processing {} % of all events'.format(int(finished)))
        k+=1
    df = df[df['prediction_label'].notna()]
    try:
        for i in range(1,4):
            df[f"prediction_label{i}"] = df["prediction_label"].apply(lambda x: x[f"Classe_{i}"])
    except KeyError:
        print("KeyError")
    if recursive:
        path_pred = "./predictions_141articles_{}_{}_{}shots_recursive.csv".format(model,prompt_method,n_shots)
    else:
        path_pred = "./predictions_141articles_{}_{}_{}shots.csv".format(model,prompt_method,n_shots)
    df.to_csv(path_pred, index=False)

# Calculs métriques 

Dictionnaires utiles

In [53]:
#Mapping pour faire coïncider les noms codés des labels avec les prédictions
mapping = {
    "Gaz à effet de serre": "gaz_effet_de_serre",
    "Elevage et utilisation des terres": "agriculture_et_utilisation_du_sol",
    "Pêche et chasse intensives": "peche_et_chasse",
    "Pollution plastique": "pollution",
    "Déforestation": "deforestation",
    "Surconsommation": "surconsommation",
    "Catastrophes naturelles": "catastrophes_naturelles",
    "Réchauffement climatique/canicules": "rechauffement_climatique_canicule",
    "Sécheresse": "secheresse",
    "Couche d'ozone": "couche_ozone",
    "Feu de forêt": "feu_foret",
    "Tensions alimentaires/famines": "tension_alim_famines",
    "Perte eau douce": "eau_potable",
    "Hausse des océans et fonte des glaces": "hausse_niveau_mer_fonte_glace",
    "Conséquence sociale": "consequence_sociale",
    "Acidification des océans": "acidification_ocean",
    "Biodiversité": "perte_biodiversite",
    "Pollution": "pollution",
    "Energie renouvelable et nucléaire": "energies_renouvelables_et_nucleaires",
    "Transports décarbonés": "transport_decarbone",
    "Engagements politiques et entreprises": "engagement_politique_et_entreprises",
    "Activisme écologique": "activisme_eco",
    "Solutions innovantes": "solution_innovante",
    "Comportement de consommation": "comportement_consommateur",
    "Reforestation": "reforestation"
}


# Cause
# Gaz à effet de serre
gaz_effet_de_serre = ["co2", "méthane", "protoxyde", "azote" "oxyde", "nitreux", "ozone", "chlorofluorocarbones", "hydrofluorocarbones", "perfluorocarbones", "soufre",
                    "hexafluorure", "gaz", "émissions", "serre", "carbone", "dioxyde", "air", "fossile"]

# Utilisation des terres
agriculture_et_utilisation_du_sol = ["agriculture", "sol", "changement", "érosion", "désertification", "dégradation", "aérosol", "pulvérisation", "bétail",
                         "agricultures", "vache", "bovin", "culture", "rendement", "récolte", "engrais", "pesticide", "labour"]

# Pêches et chasses
peche_et_chasse = ["poisson", "pêche", "chasse", "baleine", "disparition", "extinction", "espèces", "surpêche", "surchasse", "quota", "capture",
                   "prédateur"]

# Intrant chimique / pollution plastique
intrants_chimique_pollution_plastique = ["plastique", "pollution", "microplastique", "microplastiques", "plastiques", "déchets", "ordures", "détritus",
                                         "recyclage", "recycler", "chimique", "décharge", "industrielle", "contamination", "engrais"]

# Surconsommation
surconsommation = ["surconsommation", "consommation", "comportement", "achat", "compulsif", "gaspillage", "excès", "superflu", "masse", "hyperconsommation"
                   "style", "vie", "possession", "dépenser", "frénésie"]

# Déforestation
deforestation = ["déforestation", "forêt", "déboiser", "déforestier", "végétation", "arbre", "arbres", "végétal", "exploitation", "défrichement",
                 "défricher", "abattage"]

cause_thematiques = {"gaz_effet_de_serre": gaz_effet_de_serre,
                    "agriculture_et_utilisation_du_sol": agriculture_et_utilisation_du_sol,
                    "peche_et_chasse": peche_et_chasse,
                    "intrants_chimique_pollution_plastique": intrants_chimique_pollution_plastique,
                    "surconsommation": surconsommation,
                    "deforestation": deforestation}


# Conséquences
# Catastrophes naturelles
catastrophes_naturelles = ["catastrophe", "naturelle", "inondation", "catastrophique", "phénomène", "cyclone", "séisme",
                            "ouragan", "tempête", "dérégulation", "climatologie", "grêle", "vent", "typhon", "tornade", "tsunami"]

# Réchauffement climatique / canicule
rechauffement_climatique_canicule = ["réchauffement", "climatique", "chauffage", "température", "chaleur", "canicule",
                                     "chaud", "adaptation", "thermomètre", "extrême", "caniculaire"]

# Sécheresse
secheresse = ["pénurie", "aridité", "déficit", "assèchement", "sols", "manque", "precipitations", "réserves", "désert", "sécheresse", "désertique",
              "déshydratation", "irrigation", "humidité"]

# Couche d'ozone
couche_ozone = ["ozone", "appauvrissement", "trous", "chlorofluorocarbures", "Halons", "ultraviolet", "nocif", "rayonnement", "stratosphère"]

# Feu de forêt
feu_foret = ["incendies", "feu", "forêt", "pompier", "incontrôlé", "propagation", "fumée", "brulée", "étendue", "ardent", "incondescent", "pyromane"]

# Tension alimentaires / famines
tension_alim_famines = ["insécurité", "alimentaire", "famine", "malnutrition", "crise", "rareté", "besoins", "déficit", "sécurité", "affamé", "pénurie"]

# Perte d'eau douce
eau_potable = ["eau", "douce", "fraîche", "potable", "boire", "potable", "potabilité", "dégradation", "qualité", "crise", "raréfaction",
               "nappes", "phréatiques", "courante", "cristalline", "filtration", "purification", "assainissement"]

# Hausse du niveau de la mer et fontes des glaces
hausse_niveau_mer_fonte_glace = ["hausse", "mer", "océan", "océanique", "niveau", "fonte", "montée", "élévation", "côte", "marée",
                                 "bloc", "glace", "iceberg", "glacier", "permafrost", "banquise", "dégel"]

# Conséquences sociales
consequence_sociale = ["logement", "population", "déplacement", "conflit", "migration", "instabilité", "sociale", "mentale", "inégalité", "santé",
                       "mode", "vie", "pauvreté", "communauté", "vulnérabilité", "précarité", "exclusion", "dépendance", "société", "humain"]

# Acidification océan
acidification_ocean = ["acidification", "acidité", "pH", "carbonate", "calcium", "coraux", "marins", "mollusques", "coquilles", "plankton", "acidose",
                       "océan"]

# Perte de biodiversité
perte_biodiversite = ["biodiversité", "espèces", "extinction", "disparition", "animaux", "plante", "végétation", "sauvage", "menacées", "menacé",
                     "récolte", "cultures", "écologie", "marine", "terre", "réserve", "habitat", "corail", "écosystème", "conservation", 
                     "fragilité", "urgence"]

# Pollution
pollution = ["air", "eau", "sol", "émissions", "gaz", "pollution", "polluant", "déchet", "toxique", "atmosphérique", "contamination", "industriel",
             "chimique", "résidus", "dégradation", "détritus"]

consequence_thematiques = {"catastrophes_naturelles":catastrophes_naturelles,
                           "rechauffement_climatique_canicule":rechauffement_climatique_canicule,
                            "secheresse":secheresse,
                            "couche_ozone":couche_ozone,
                            "feu_foret":feu_foret,
                            "tension_alim_famines":tension_alim_famines,
                            "eau_potable":eau_potable,
                            "hausse_niveau_mer_fonte_glace":hausse_niveau_mer_fonte_glace,
                            "consequence_sociale":consequence_sociale,
                            "acidification_ocean":acidification_ocean,
                            "perte_biodiversite":perte_biodiversite,
                            "pollution":pollution}


# Solutions
# Énergies renouvelables et nucléaires
energies_renouvelables_et_nucleaires = ["renouvelable", "solaire", "éolien", "biomasse", "géothermique", "hydraulique", "hydroélectrique", "photovoltaïque",
                                "durabilité", "transition", "nucléaire", "propre", "thermique", "durable", "électricité", "électrique", "puissance",
                                "stockage", "fission", "fusion"]

# Transport décarbonés
transport_decarbone = ["véhicule", "électrique", "batterie", "commun", "transport", "hybride", "covoiturage", "vélo", "hydrogène", "aménagement", "mobilité",
                       "routier", "ferroviaire", "électrification"]

# Engagement politique et entreprises
engagement_politique_et_entreprises = ["gouvernance", "législation", "accord", "engagement", "stratégie", "diplomatie", "acteurs", "lobbying", "participation",
                                     "responsabilité", "traités", "financement", "Paris", "régulation", "organisations", "politique", "économie",
                                     "projet", "loi", "certification", "investissement"]

# Activisme écologique
activisme_eco = ["militantisme", "mouvement", "écologie", "marches", "protestation", "défense", "environnement", "activisme", "engagement", "protection",
                 "grève", "boycott", "mobilisation", "plaidoyer", "désobéissance", "civile", "dénonciation", "dénoncer", "action", "manifestation"]

# Solutions innovantes
solution_innovante = ["technologie", "verte", "innovation", "développement", "produit", "nouvelle", "approche", "technologie", "propre", "recherche",
                      "écoconception", "circulaire", "créativité", "créatif", "alternatif", "solution"]

# Comportement de consommation
comportement_consommateur = ["consommateur", "consommation", "comportement", "style", "vie", "responsable", "durable", "minimalisme", "consciente",
                             "équitable", "éthique", "locaux", "collaborative", "collectivité", "astuce"]

# Reforestation
reforestation = ["reboisement", "plantation", "replantation", "arbre", "reforestation", "carbone", "lutte", "régénération", "régénérer", "restauration",
                 "forêt", "biodiversité", "reconstitution", "sylviculture", "boisement", "repeuplement", "repeupler"]

solution_thematiques = {"energies_renouvelables_et_nucleaires": energies_renouvelables_et_nucleaires,
                        "transport_decarbone": transport_decarbone,
                        "engagement_politique_et_entreprises": engagement_politique_et_entreprises,
                        "activisme_eco": activisme_eco,
                        "solution_innovante": solution_innovante,
                        "comportement_consommateur": comportement_consommateur,
                        "reforestation": reforestation}


Fonctions utiles

In [54]:
def format_labels(labels, mapping):
    """
    Formate les labels en utilisant le mapping, mais conserve les labels déjà formatés.
 
    Arguments :
    - labels : list, la liste des labels à formater.
    - mapping : dict, le dictionnaire de correspondance des labels.

    Retourne :
    - list, la liste des labels formatés.
    """
    formatted_labels = []
    for label in labels:
        # Si le label est déjà dans le bon format (valeur du mapping), on le conserve
        if label in mapping.values():
                formatted_labels.append(label)
        # Sinon, on utilise le mapping pour le formater
        else:
            formatted_label = mapping.get(label, "N/A")
            if formatted_label != "N/A":
                formatted_labels.append(formatted_label)
    return formatted_labels

def get_perf(df_results, pred_col):

    y_true = df_results["label"]
    mlb = MultiLabelBinarizer()
    full_dic = cause_thematiques | consequence_thematiques | solution_thematiques
    mlb.fit([pd.Series(list(full_dic.keys())).unique()])
    mlb.classes_

    y_true_binarized = mlb.transform(y_true)
    y_pred = df_results[pred_col]
    y_pred_binarized = mlb.transform(y_pred)
    
    def top3_accuracy(preds, labels):
        for pred in preds:
            if pred in labels:
                return 1
        return 0

    def top1_accuracy(preds, labels):
        if len(preds) == 0:
            return 0
        
        elif preds[0] in labels:
            return 1
        return 0

    df_score_global = pd.DataFrame({"Precision": [precision_score(y_true_binarized, y_pred_binarized, average='weighted')*100],
                                              
                                "Recall": [recall_score(y_true_binarized, y_pred_binarized, average='weighted')*100],
                                              
                                "F1_score": [f1_score(y_true_binarized, y_pred_binarized, average='weighted')*100],
                                "Top 1 Accuracy": [round(df_results.apply(lambda x: top1_accuracy(x[pred_col], x["label"]), axis=1).sum()/len(df_results)*100,2)],
                                "Top 3 Accuracy": [round(df_results.apply(lambda x: top3_accuracy(x[pred_col], x["label"]), axis=1).sum()/len(df_results)*100,2)],
                                "Percentage of unsure": [round(df_results.apply(lambda x: 1 if len(x[pred_col])==0 else 0, axis=1).sum()/len(df_results)*100,2)]                     
                                })

    return df_score_global.round(2)

Sorties

In [57]:
modèles = ["ministral8b"]
prompt_methods = ["CARP"]
number_shots = [1]
paths = []
for model in modèles:
    for prompt_method in prompt_methods:
        for n_shot in number_shots:
            get_df_pred(path_df = path_df,classification = classification_up_to_k_general,prompt_method = prompt_method,model = model,n_shots= n_shot)
            paths.append("./predictions_141articles_{}_{}_{}shots.csv".format(model,prompt_method,n_shot))
            print(f"Done_{model}_{prompt_method}_{n_shot}shots")
            

Unexpected error: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant
Finished processing 19 % of all events
Finished processing 39 % of all events
Finished processing 59 % of all events
Finished processing 79 % of all events
Finished processing 99 % of all events
KeyError
Done_ministral8b_CARP_1shots


In [ ]:
def pipe(path):
        df = pd.read_csv(path)
 
        # Create a copy to avoid SettingWithCopyWarning
           
        df_results = df[
            ["description", "label_1", "label_2", "label_3",
            "prediction_label_1", "prediction_label_2", "prediction_label_3"]
        ].copy()
        # Format true labels
        df_results["label"] = df_results[["label_1", "label_2", "label_3"]].apply(
            lambda x: format_labels(x.dropna().tolist(), mapping), axis=1
        )
        # Format predicted labels
        df_results["pred_label"] = df_results[
            ["prediction_label_1", "prediction_label_2", "prediction_label_3"]
        ].apply(
            lambda x: format_labels(x.dropna().tolist(), mapping), axis=1
        )
        # Affiche les labels bruts avant de les passer à format_labels
 
 
        # Count hallucinations (N/A in predictions)
        counter = sum(1 for preds in df_results["pred_label"] if "N/A" in preds)
        print(f"Nombre de documents avec une hallucination: {counter}")
        return get_perf(df_results, "pred_label")


In [ ]:
paths.append("./predictions_141articles_ministral8b_COT_0shots.csv")
paths.append("./predictions_141articles_ministral8b_CARP_0shots.csv")
paths.append("./predictions_141articles_ministral8b_COT_1shots.csv")

In [94]:
paths = list(set(paths))
paths

['./predictions_141articles_ministral8b_COT_1shots.csv',
 './predictions_141articles_ministral8b_CARP_1shots.csv',
 './predictions_141articles_ministral8b_CARP_0shots.csv',
 './predictions_141articles_ministral8b_COT_0shots.csv']

In [95]:
import ast
for path in paths:
    df = pd.read_csv(path)
    df['prediction_label_1'] = df["prediction_label"].apply(lambda x: ast.literal_eval(x)['Classe_1'])
    df['prediction_label_2'] = df["prediction_label"].apply(lambda x: ast.literal_eval(x)["Classe_2"])
    df['prediction_label_3'] = df["prediction_label"].apply(lambda x: ast.literal_eval(x)["Classe_3"])
    df2 = df.T.drop_duplicates().T
    df.to_csv(path, index=False)
path = "./predictions_141articles_ministral8b_COT_1shots.csv"


In [96]:
import pandas as pd
import os

result = pd.DataFrame()

for path in paths:
    print(path)
    df_result = pipe(path)  
    
    if df_result is not None and not df_result.empty:  
        filename = os.path.basename(path)  
        index_name = "_".join(filename.replace(".csv", "").split("_")[-3:]) 

        df_result["source"] = index_name  
        result = pd.concat([result, df_result], ignore_index=True)

result.set_index("source", inplace=True)

new_data = pd.DataFrame([[51.64, 54.25, 46.51, 63.04, 81.88, 1.45]], 
                        columns=result.columns, 
                        index=["chatgpt_ancien"])

result = pd.concat([result, new_data])

result


./predictions_141articles_ministral8b_COT_1shots.csv
Nombre de documents avec une hallucination: 0
./predictions_141articles_ministral8b_CARP_1shots.csv
Nombre de documents avec une hallucination: 0
./predictions_141articles_ministral8b_CARP_0shots.csv
Nombre de documents avec une hallucination: 0
./predictions_141articles_ministral8b_COT_0shots.csv
Nombre de documents avec une hallucination: 0


,Precision,Recall,F1_score,Top 1 Accuracy,Top 3 Accuracy,Percentage of unsure
ministral8b_COT_1shots,51.97,60.23,50.83,61.70,87.23,0.00
ministral8b_CARP_1shots,50.06,56.87,47.07,63.57,82.86,0.00
ministral8b_CARP_0shots,53.87,53.41,44.30,54.61,82.27,0.00
ministral8b_COT_0shots,52.30,54.92,44.60,61.70,85.82,0.00
chatgpt_ancien,51.64,54.25,46.51,63.04,81.88,1.45
